In [1]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [2]:
from rpy2.robjects import r
r.data('iris')
df_iris = pandas2ri.ri2py(r.data('iris'))

In [3]:
import pandas.rpy.common as com
infert = com.load_data('infert')
infert.head()

/Users/yuewang/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html
  if __name__ == '__main__':


,education,age,parity,induced,case,spontaneous,stratum,pooled.stratum
1,0-5yrs,26.0,6.0,1.0,1.0,2.0,1,3.0
2,0-5yrs,42.0,1.0,1.0,1.0,0.0,2,1.0
3,0-5yrs,39.0,6.0,2.0,1.0,0.0,3,4.0
4,0-5yrs,34.0,4.0,2.0,1.0,0.0,4,2.0
5,6-11yrs,35.0,3.0,1.0,1.0,1.0,5,32.0


In [4]:
from numpy import *
import scipy as sp
import pandas as pd
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import pandas.rpy.common as com

In [5]:
stats = importr('stats')
base = importr('base')
datasets = importr('datasets')
graph = importr('graphics')
forecast = importr('forecast')
fortify = importr('ggfortify')

In [6]:
ro.r('library(forecast)')

array(['ggfortify', 'ggplot2', 'forecast', 'timeDate', 'zoo', 'tools',
       'stats', 'graphics', 'grDevices', 'utils', 'datasets', 'methods',
       'base'], 
      dtype='|S9')

In [7]:
count = range(1, 51)
df = pd.DataFrame(index = pd.date_range('2016-01-01', '2016-02-19'), data = count, columns = ['count'])
# df['count'] = count
# df['date'] = pd.date_range('2016-01-01', '2016-02-19')
# df.set_index('date', inlace = True)
# df.sort_index(inplace = True)
pandas2ri.activate()
r_timeseries = pandas2ri.py2ri(df)

In [17]:
df.shape

(50, 1)

In [8]:
ro.r('library(forecast)')

array(['ggfortify', 'ggplot2', 'forecast', 'timeDate', 'zoo', 'tools',
       'stats', 'graphics', 'grDevices', 'utils', 'datasets', 'methods',
       'base'], 
      dtype='|S9')

In [16]:
df

,count
2016-01-01,1
2016-01-02,2
2016-01-03,3
2016-01-04,4
2016-01-05,5
2016-01-06,6
2016-01-07,7
2016-01-08,8
2016-01-09,9
2016-01-10,10


In [10]:
rdf = pandas2ri.py2ri(df)
ro.globalenv['r_timeseries'] = rdf
pred = ro.r('as.data.frame(forecast(auto.arima(r_timeseries),h=5))')

In [11]:
pred

,Point Forecast,Lo 80,Hi 80,Lo 95,Hi 95
51,51.0,51.0,51.0,51.0,51.0
52,52.0,52.0,52.0,52.0,52.0
53,53.0,53.0,53.0,53.0,53.0
54,54.0,54.0,54.0,54.0,54.0
55,55.0,55.0,55.0,55.0,55.0


In [ ]:
pred1 = ro.r('as.data.frame(forecast(auto.arima(r_timeseries), h=10))')
pred1

In [15]:
ro.globalenv['r_timeseries']

R object with classes: ('data.frame',) mapped to:
<DataFrame - Python:0x11d0c2f38 / R:0x11c862518>
[IntVector]
  count: <class 'rpy2.robjects.vectors.IntVector'>
  R object with classes: ('integer',) mapped to:
<IntVector - Python:0x11d0db0e0 / R:0x11a1057b0>
[       1,        2,        3, ...,       48,       49,       50]

In [ ]:
df = com.load_data('mtcars')

In [ ]:
rdf = com.convert_to_r_dataframe(df)

In [ ]:
formula = 'mpg ~ wt + cyl'

In [ ]:
fit_full = stats.lm(formula, data=rdf)
# print(base.summary(fit_full))

In [ ]:
volcanodust = ro.r.scan("http://robjhyndman.com/tsdldata/annual/dvi.dat", skip=1)

In [ ]:
volcanodustseries = ro.r.ts(volcanodust,start=ro.r.c(1500))

In [ ]:
# re = stats.pacf(volcanodustseries)

In [32]:
import numpy as np
count = np.random.rand(214,1)
df = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-08-01'), data=count, columns=['count'])

In [36]:
# df['count'].dropna()
df['count'].dropna().shape
df

,count
2016-01-01,0.263964
2016-01-02,0.855177
2016-01-03,0.868950
2016-01-04,0.367304
2016-01-05,0.795551
2016-01-06,0.567739
2016-01-07,0.548149
2016-01-08,0.473667
2016-01-09,0.423687
2016-01-10,0.515494


In [39]:
rdf = pandas2ri.py2ri(df['count'].dropna())
ro.globalenv['r_timeseries'] = rdf
pred = ro.r('as.data.frame(forecast(auto.arima(r_timeseries),h=1))')

In [38]:
pred

,Point Forecast,Lo 80,Hi 80,Lo 95,Hi 95
215,0.491939,0.13411,0.849767,-0.055312,1.03919


In [29]:
type(pred)

pandas.core.frame.DataFrame

In [46]:
df.shape[1]

1

In [53]:
len(df['count'].shape)

1

In [54]:
len(df.shape)

2

In [71]:
count = np.random.rand(214,2)
df = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-08-01'), data=count, columns=['count1','count2'])

In [75]:
df.dtypes

count1    float64
count2    float64
dtype: object

In [79]:
df[['count1']].dropna().dtypes

count1    float64
dtype: object

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from BL_model_v2 import *
from BL_data_reader import *

In [ ]:
def FACTOR_EMA(df, half_life):
        return pd.ewma(df, halflife=half_life)

In [ ]:
view_config = {"MA5" : lambda df: FACTOR_EMA(df.shift(1).dropna(axis=0), 5),
"MA30" : lambda df: FACTOR_EMA(df.shift(1).dropna(axis=0), 30),
"MA120" : lambda df: FACTOR_EMA(df.shift(1).dropna(axis=0), 120)}


In [ ]:
DR_view = BLDR.view_data_reader("/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/factor_return_w_industry.csv", view_config,                            error_periods=60, error_method = "rolling_window")
DR_factor_return = BLDR.factor_return_data_reader("/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/factor_return_w_industry.csv",
                                                  look_back_periods=60)
DR_residual = BLDR.residual_data_reader("/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/residual.csv",
                                        look_back_periods=60, threshold = None, min_periods=2, diagonalized=True)
DR_residual.set_threshold(0.000001)
DR_factor_loading = BLDR.factor_loading_data_reader("/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/factor_loading_w_industry.csv")

In [ ]:
data_reader_config = {'view': DR_view, #call method return estimation and standard error of each predictors
        'factor_return': DR_factor_return, #call method return prior mean and covariance of factors
            'residual': DR_residual,#call method return covariance of residuals
                'factor_loading': DR_factor_loading #call method return factor loadings of each stock, also return the return of next B day for each stock
                }
BLP = Black_Litterman_Portfolio(data_reader_config, 1, factor_diagonalized=False)
print BLP.view.methods

In [ ]:
target_date = datetime(2015,5,8)
EST, ERR = BLP.view('MA5',target_date)

In [ ]:
EST

In [ ]:
ERR

In [ ]:
file_dir = "/Users/yuewang/Dropbox/Capstone Project/structured_code_and_data/Data/factor_return_w_industry.csv"
_df_factor_return = pd.read_csv(file_dir, sep =',',index_col=0)
_df_factor_return.index = _df_factor_return.index.map(lambda idx: datetime.strptime(idx,"%Y-%m-%d"))
_df_factor_return.sort_index(inplace=True)

In [ ]:
import pickle
pickle.dump(_df_factor_return, open("factor_return.p", "wb" ) )

In [ ]:
_df_factor_return.iloc[:100,:].BETA

In [ ]:
r_tmp = pandas2ri.py2ri(_df_factor_return.iloc[:200,:].BETA)

In [ ]:
ro.globalenv['tmp'] = r_tmp

In [ ]:
base.rm('tmp') ## Get rid of that global variables

In [1]:
pred = ro.r('as.data.frame(forecast(auto.arima(tmp),h=1))')

NameError: name 'ro' is not defined

In [ ]:
# _df_factor_return.iloc[:100,:].BETA
pred

In [ ]:
pred['Point Forecast'][0]

In [ ]:
# %matplotlib inline

In [ ]:
# _df_factor_return.iloc[:100,:].BETA.plot()

In [ ]:
#### use holt winter

In [ ]:
# (m <- HoltWinters(AirPassengers, seasonal = "mult"))

In [ ]:
# m = ro.r('as.data.frame(forecast(HoltWinters(tmp),12))')

In [ ]:
ro.globalenv['tmp']

In [ ]:
### have to convert tmp to a ts

In [ ]:
data_pd = _df_factor_return.iloc[200,:].BETA

In [ ]:
data_pd.index[0], data_pd.index[-1] 

In [ ]:
data_pd.index[0].year, data_pd.index[0].month, data_pd.index[0].day

In [ ]:
# ts1 = stats.ts(r_tmp, start=ro.r.c(data_pd.index[0].year, data_pd.index[0].month, data_pd.index[0].day
# ), end=ro.r.c(data_pd.index[1].year, data_pd.index[1].month, data_pd.index[1].day), frequency=365) 

In [ ]:
ts1 = stats.ts(r_tmp, start=1, frequency=356) 

In [ ]:
ro.globalenv['ts1'] = ts1

In [ ]:
# hw <- HoltWinters(demand)
m = ro.r('as.data.frame(forecast(HoltWinters(ts1,gamma = FALSE),1))')

In [ ]:
m

In [ ]:
##### VAR prediction

In [ ]:
vars = importr("vars")

In [ ]:
data_pd_var.index

In [ ]:
data_pd_var = _df_factor_return.iloc[:200,:]
r_tmp_var = pandas2ri.py2ri(data_pd_var)

In [ ]:
ts1_var = stats.ts(r_tmp_var, start=1, frequency=356) 

In [ ]:
ro.globalenv['ts1_var'] = ts1_var

In [ ]:
a = 2

In [ ]:
"as.data.frame(fortify(predict(VAR(ts1_var,AAA), n.ahead = 1)))".replace("AAA", str(a))

In [ ]:
m1 = ro.r("as.data.frame(fortify(predict(VAR(ts1_var,AAA), n.ahead = 1)))".replace("AAA", str(a)))

In [ ]:
m = ro.r('as.data.frame(fortify(predict(VAR(ts1_var,1), n.ahead = 1)))')

In [ ]:
ro.globalenv['m'] = m

In [ ]:
pydf = com.load_data('m')
pydf.columns

In [ ]:
pydf = pydf.filter(regex='fcst')
pydf

In [ ]:
ts1 = stats.ts(r_tmp, start=1, frequency=356) 

In [ ]:


_df_factor_return.iloc[:200,:]

In [ ]:
tmp = pd.DataFrame(index=_df_factor_return.index, columns=_df_factor_return.columns)

In [ ]:
tmp.iloc[:50, :] = _df_factor_return.iloc[:50, :]

In [ ]:
tmp.shape

In [ ]:
for col in tmp.columns:
    for row in tmp.iloc[2510:,:].index:
#         print (row, col)
        tmp.ix[row,[col]] = 2
tmp.ix[row,[col]]

In [ ]:
count = np.random.rand(100,1)
df = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-04-09'), data=count, columns=['count'])

In [ ]:
df

In [ ]:
tmp.ix[row,[col]]

In [ ]:
tmp.tail

In [ ]:
from statsmodels.tsa.arima_process import arma_generate_sample
import statsmodels.api as sm
import numpy as np

In [ ]:
res = sm.tsa.arma_order_select_ic(np.array(_df_factor_return.BETA), ic=['aic', 'bic'], trend='nc')

In [ ]:
raw_predictor = sm.tsa.ARMA(_df_factor_return.BETA, res.aic_min_order).fit()

In [ ]:
sm.tsa.x13_arima_select_order()

In [ ]:
res1 = sm.tsa.arima_order_select_ic(np.array(_df_factor_return.iloc[:100,:].BETA), ic=['aic', 'bic'], trend='nc')

In [ ]:
raw_predictor1 = sm.tsa.ARMA(_df_factor_return.iloc[:100,:].BETA, res1.bic_min_order).fit()

In [ ]:
_df_factor_return.iloc[:100,:].BETA

In [ ]:
import datetime
datetime.date(2006, 5, 26)

In [ ]:
raw_predictor1.predict('20060526','20060527', dynamic=True)

In [ ]:
predict_sunspots = raw_predictor1.predict('1990', '2012', dynamic=True)

In [ ]:
_df_factor_return.columns

In [ ]:
len(np.array(_df_factor_return.BETA))

In [ ]:
len(np.array(_df_factor_return.iloc[:100,:].BETA))

In [ ]:
res.aic_min_order

In [ ]:
res.bic_min_order

In [ ]:
_df_factor_return

In [ ]:
dta = sm.datasets.sunspots.load_pandas().data

In [ ]:
dta

In [ ]:
import pandas

In [ ]:
mdata = sm.datasets.macrodata.load_pandas().data

In [ ]:
dates = mdata[['year', 'quarter']].astype(int).astype(str)

In [ ]:
quarterly = dates["year"] + "Q" + dates["quarter"]

In [ ]:
from statsmodels.tsa.base.datetools import dates_from_str

In [ ]:
quarterly = dates_from_str(quarterly)

In [ ]:
mdata = mdata[['realgdp','realcons','realinv']]

In [ ]:
mdata.index = pandas.DatetimeIndex(quarterly)

In [ ]:
data = np.log(mdata).diff().dropna()

In [ ]:
model = VAR(data)

In [ ]:
from __future__ import print_function
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.graphics.api import qqplot

In [ ]:
print(sm.datasets.sunspots.NOTE)


In [ ]:
dta = sm.datasets.sunspots.load_pandas().data

In [ ]:
dta.index = pd.Index(sm.tsa.datetools.dates_from_range('1700', '2008'))
del dta["YEAR"]